In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df['text'][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It By Darrell Lucus on October 30, 2016 Subscribe Jason Chaffetz on the stump in American Fork, Utah ( image courtesy Michael Jolley, available under a Creative Commons-BY license) \nWith apologies to Keith Olbermann, there is no doubt who the Worst Person in The World is this week–FBI Director James Comey. But according to a House Democratic aide, it looks like we also know who the second-worst person is as well. It turns out that when Comey sent his now-infamous letter announcing that the FBI was looking into emails that may be related to Hillary Clinton’s email server, the ranking Democrats on the relevant committees didn’t hear about it from Comey. They found out via a tweet from one of the Republican committee chairmen. \nAs we now know, Comey notified the Republican chairmen and Democratic ranking members of the House Intelligence, Judiciary, and Oversight committees that his agency was reviewing emai

In [5]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [6]:
df=df.dropna()

In [7]:
X = df.drop('label',axis=1)
y= df['label']
X.shape

(18285, 4)

In [8]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [9]:
voc_size=5000

In [10]:
messages=X.copy()

In [11]:
messages.reset_index(inplace=True)

In [12]:
import nltk
import re
from nltk.corpus import stopwords

In [13]:
#nltk.download('stopwords')

In [14]:
messages.head()

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [15]:
X.shape

(18285, 4)

In [16]:
X = X[~X['title'].str.isnumeric()]


In [17]:
X.shape

(18285, 4)

In [18]:
print(messages.columns)


Index(['index', 'id', 'title', 'author', 'text'], dtype='object')


In [19]:
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if word not in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [20]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr[1:5]

[[3866, 578, 3813, 1565, 802, 772, 4582],
 [970, 325, 3327, 3308],
 [3724, 2366, 531, 490, 538, 3367],
 [4878, 802, 1289, 1866, 4138, 4840, 802, 773, 713, 2092]]

One-hot encoding is a simple way to represent words numerically, but it doesn't capture semantic similarities between words.

<h3>Using Word Embeddings and LSTM
<h4>Add Word Embeddings just add zero in the beginning or the End


In [21]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs[1:10])

[[3866  578 3813 1565  802  772 4582    0    0    0    0    0    0    0
     0    0    0    0    0    0]
 [ 970  325 3327 3308    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0]
 [3724 2366  531  490  538 3367    0    0    0    0    0    0    0    0
     0    0    0    0    0    0]
 [4878  802 1289 1866 4138 4840  802  773  713 2092    0    0    0    0
     0    0    0    0    0    0]
 [1730 3492 1993 4963 3749 1785 4576 4818 2034  358 1106  925 1246  367
  4582    0    0    0    0    0]
 [1931 1585 4078  394 1813 2876  216 4321 4220 1968 2318    0    0    0
     0    0    0    0    0    0]
 [2578 4255 4899 1191 3246 1935 1785 4073 4220 1968 2318    0    0    0
     0    0    0    0    0    0]
 [2276 4007 2346 2237 4576 4710  186 3284 1785  772    0    0    0    0
     0    0    0    0    0    0]
 [4213 3428 4855 3359 3853 1559 1825 3869    0    0    0    0    0    0
     0    0    0    0    0    0]]


In [22]:
embedded_vector_features=40
model=Sequential()

<h4>Applying LSTM on the Data 

In [23]:
model.add(Embedding(voc_size,embedded_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
# new_learning_rate = 0.001
# model.compile(optimizer=SGD(learning_rate=new_learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
print(model.summary())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [24]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [25]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [26]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [ ]:
hjkl

In [27]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10


KeyboardInterrupt: 

<h3>Adding Dropout Layer to the Model

In [ ]:
from tensorflow.keras.layers import Dropout
import tensorflow as tf
## Creating model
embedding_vector_features=40
model2=Sequential()
model2.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model2.add(Dropout(0.3))
model2.add(LSTM(100))
model2.add(Dropout(0.3))
model2.add(Dense(1,activation='sigmoid'))
model2.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
early_stopping=tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0.001,
    patience= 20,
    verbose=1,
    mode="auto",
    baseline=None,
    restore_best_weights=False,
)

In [ ]:
### Finally Training
model_history=model2.fit(X_train,y_train,validation_split=0.33,batch_size=32,epochs=30,callbacks=early_stopping)

Epoch 1/30
257/257 [==============================] - 12s 31ms/step - loss: 0.3096 - accuracy: 0.8531 - val_loss: 0.1968 - val_accuracy: 0.9159
Epoch 2/30
257/257 [==============================] - 7s 29ms/step - loss: 0.1371 - accuracy: 0.9491 - val_loss: 0.2295 - val_accuracy: 0.9048
Epoch 3/30
257/257 [==============================] - 7s 28ms/step - loss: 0.0890 - accuracy: 0.9701 - val_loss: 0.2436 - val_accuracy: 0.9129
Epoch 4/30
257/257 [==============================] - 7s 27ms/step - loss: 0.0613 - accuracy: 0.9794 - val_loss: 0.2994 - val_accuracy: 0.9040
Epoch 5/30
257/257 [==============================] - 8s 30ms/step - loss: 0.0505 - accuracy: 0.9831 - val_loss: 0.3160 - val_accuracy: 0.9100
Epoch 6/30
257/257 [==============================] - 8s 31ms/step - loss: 0.0350 - accuracy: 0.9899 - val_loss: 0.3831 - val_accuracy: 0.8998
Epoch 7/30
257/257 [==============================] - 8s 31ms/step - loss: 0.0338 - accuracy: 0.9889 - val_loss: 0.4588 - val_accuracy: 0.890

In [ ]:
y_pred=model2.predict(X_test)

189/189 [==============================] - 1s 5ms/step


In [ ]:
y_pred=np.where(y_pred > 0.5, 1,0) ##AUC ROC Curve
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(y_test,y_pred)

array([[3111,  308],
       [ 293, 2323]], dtype=int64)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.91      0.91      0.91      3419
           1       0.88      0.89      0.89      2616

    accuracy                           0.90      6035
   macro avg       0.90      0.90      0.90      6035
weighted avg       0.90      0.90      0.90      6035



<h3>Using Tensorflow's Word2vec

<h3>Additional Tasks to perform on this project
<h5>->In the Neural Network try using different Optimizers along with changing learning rate to see its effect on accuracy
<h5>->Increase the number of epochs and apply a limit to it to get the best accuracy  


<h3> Using Distill BERT

In [31]:
import tensorflow_text as text
import tensorflow as tf
import tensorflow_hub as hub
# Input layer for text data
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)

# BERT preprocessor layer (distilBERT)
preprocessor = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3",
    name="bert_preprocessor"
)
encoder_inputs = preprocessor(text_input)

# Smaller BERT encoder layer (distilBERT)
encoder = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1",
    trainable=True,
    name="bert_encoder"
)
outputs = encoder(encoder_inputs)
pooled_output = outputs["pooled_output"]

ModuleNotFoundError: No module named 'tensorflow_text'